In [1]:
!pip install earthengine-api #earth-engine Python API

In [2]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Lcl1Lzd-jZNKyvgq94rShkNqGBZ1lA__gHNiOBXwiSk&tc=mAu_0f-9xvE_lFcu8gyWXbvA0rSYK8e2jIIaD_nFMPA&cc=EAEonqbMjZzX6LuyKBdapdmnke9oAhlH5w1JMR45N4A

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkLNKfkApKvCHsnYEPPq6uyQzcUj3J2bAYJy7z_D4jdBs_BFVy_W0o

Successfully saved authorization token.


In [3]:
import ee
import datetime

ee.Initialize()

In [4]:
location={
      "type": "Feature",
      "properties": {
        "name": "test"
      },
      "geometry": {
        "coordinates": [
          [
            [
              51.75889330279665,
              35.491301434102795
            ],
            [
              51.75776151360827,
              35.492283033293916
            ],
            [
              51.75573167430704,
              35.490740514895094
            ],
            [
              51.75614994422264,
              35.49032983942088
            ],
            [
              51.75420621931508,
              35.48912785036225
            ],
            [
              51.75494434269825,
              35.48828644732369
            ],
            [
              51.75792144034125,
              35.48972884713936
            ],
            [
              51.75889330279665,
              35.491301434102795
            ]
          ]
        ],
        "type": "Polygon"
      },
      "id": 1
    }
fdate='2022-12-22'
edate='2023-02-22'

In [5]:
if 'geometry' in location:
  geometry = ee.Geometry(location['geometry'])
else:
  geometry = ee.Geometry(location)

find date series of satellite

In [6]:
image_colection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(fdate,edate).filterBounds(geometry)
dates_miliseconds = image_colection.reduceColumns(ee.Reducer.toList(), ["system:time_start"]).get('list').getInfo()
cloudy_pixel_precentage = image_colection.aggregate_array("CLOUDY_PIXEL_PERCENTAGE").getInfo()

In [7]:
dates = [datetime.datetime.fromtimestamp(mili_second/1000).strftime("%Y-%m-%d") for mili_second in dates_miliseconds]
dates


['2022-12-23',
 '2022-12-28',
 '2023-01-02',
 '2023-01-07',
 '2023-01-12',
 '2023-01-17',
 '2023-01-22',
 '2023-01-27',
 '2023-02-01',
 '2023-02-06',
 '2023-02-11',
 '2023-02-16',
 '2023-02-21']

In [8]:
cloudy_pixel_precentage

[3.141383,
 17.800988,
 0.000322,
 3.146922,
 66.381991,
 62.436754,
 7.373483,
 48.20841,
 4.202946,
 51.896495,
 23.116735,
 11.311141,
 0.113497]

start  get image

In [52]:
bandIndexExps = [
                '(b(8) - b(4)) / (b(8) +b(4))',
                '(b(7) - b(4)) / (b(7) +b(4))',
                '((2 * b(7) + 1 - sqrt(((2 * b(7) + 1) * 2) - 8*(b(7) - b(3)))) / 2)',
                '(b(7) - b(10)) / (b(7) +b(10))',
                '(b(11) - b(7)) / (b(11) +b(7))',
                'b(7) / (b(3) + b(7))',
                'b(7) - b(3)',
                '(2.5 * (b(7) - b(3))) / (1 + b(7) + 6* b(3) - 7.5* b(2))',
                '1.16 * ((b(7) - b(3)) / (b(7) + b(3) + 0.16))',
                'b(3) - 0.735 * (b(1) - b(3))',
                '(2 * (b(7)**2 -b(3)**2) + 1.5*b(7) + 0.5*b(3)) / (b(7) + b(3) + 0.5)'
                ]
NDVI =None
bandIndexNames =  ['NDVI', 'NDRE', 'MSAVI2', 'NDMI', 'NDWI', 'IPVI', 'DVI', 'SARVI2', 'OSAVI','RGB','n']


params = {
  "NDVI":{ 'min': -1,
            'max': 1,
            'palette': [
                    "#007e47", "#009755", "#14aa60", "#77ca6f", "#b9e383",
                    "#d5ef94", "#eaf7ac", "#fdfec2", "#ffefab", "#ffe093","#ffc67d",
                    "#ffab69", "#ff8d5a", "#ffcc66", "#fe6c4a", "#ef4c3a", "#e02d2c",
                    "#c5142a", "#ad0028"]
          },
  'NDRE':{ 'min': -1,
            'max': 1,
            'palette': [
                    "#800000", "#800000", "#800000", "#800000", "#800000",
                    "#800000", "#800000", "#800000", "#800000", "#800000","#800000",
                    "#ff6600", "#ff9966", "#ffcc66", "#ccff99", "#ccff99", "#99ff99",
                    "#00ff00", "#009933", "#003300"]
          },
  'MSAVI2':{ 'min': -1,
            'max': 1,
            'palette': [
                    "#800000", "#800000", "#800000", "#800000", "#800000",
                    "#800000", "#800000", "#800000", "#800000", "#800000","#800000",
                    "#ff6600", "#ff9966", "#ffcc66", "#ccff99", "#ccff99", "#99ff99",
                    "#00ff00", "#009933", "#003300"]
          },
  'NDMI':{ 'min': -1,
            'max': 1,
            'palette': [
                    "#800000", "#800000", "#800000", "#800000", "#800000",
                    "#800000", "#800000", "#800000", "#800000", "#800000","#800000",
                    "#ff6600", "#ff9966", "#ffcc66", "#ccff99", "#ccff99", "#99ff99",
                    "#00ff00", "#009933", "#003300"]
          },
  'NDWI':{ 'min': -1,
            'max': 1,
            'palette': [
                    "#800000", "#800000", "#800000", "#800000", "#800000",
                    "#800000", "#800000", "#800000", "#800000", "#800000","#800000",
                    "#ff6600", "#ff9966", "#ffcc66", "#ccff99", "#ccff99", "#99ff99",
                    "#00ff00", "#009933", "#003300"]
          },
  'IPVI':{ 'min': -1,
            'max': 1,
            'palette': [
                    "#800000", "#800000", "#800000", "#800000", "#800000",
                    "#800000", "#800000", "#800000", "#800000", "#800000","#800000",
                    "#ff6600", "#ff9966", "#ffcc66", "#ccff99", "#ccff99", "#99ff99",
                    "#00ff00", "#009933", "#003300"]
          },
  'DVI':{ 'min': -1,
            'max': 1,
            'palette': [
                    "#800000", "#800000", "#800000", "#800000", "#800000",
                    "#800000", "#800000", "#800000", "#800000", "#800000","#800000",
                    "#ff6600", "#ff9966", "#ffcc66", "#ccff99", "#ccff99", "#99ff99",
                    "#00ff00", "#009933", "#003300"]
          },
  'SARVI2':{ 'min': -1,
            'max': 1,
            'palette': [
                    "#800000", "#800000", "#800000", "#800000", "#800000",
                    "#800000", "#800000", "#800000", "#800000", "#800000","#800000",
                    "#ff6600", "#ff9966", "#ffcc66", "#ccff99", "#ccff99", "#99ff99",
                    "#00ff00", "#009933", "#003300"]
          },
  'RGB':{ 'min': 0,
            'max': 6500,
            'palette': []
          },
  'OSAVI':{ 'min': -1,
            'max': 1,
            'palette': [
                    "#800000", "#800000", "#800000", "#800000", "#800000",
                    "#800000", "#800000", "#800000", "#800000", "#800000","#800000",
                    "#ff6600", "#ff9966", "#ffcc66", "#ccff99", "#ccff99", "#99ff99",
                    "#00ff00", "#009933", "#003300"]
          }
  }

In [84]:
bandName="NDVI"
fdate='2023-01-02'
edate='2023-01-04'


In [85]:
IndexNames=bandIndexNames.index(bandName)

if 'geometry' in location:
  geometry = ee.Geometry(location['geometry'])
else:
  geometry = ee.Geometry(location)

In [86]:
image_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(fdate,edate).filterBounds(geometry)
image_collection_size = image_collection.size().getInfo()
image_collection_list = image_collection.toList(image_collection_size)
selected_image_index = 0

image = ee.Image(image_collection_list.get(selected_image_index)).clip(geometry)

image_collection_size


1

In [93]:
ndvi=image.expression(
"(NIR - RED) / (NIR + RED)",
{"NIR":image.select("B8"),
"RED":image.select("B4")});
ndvi=ndvi.rename(str(bandName))

ndvivis={
    'bands': ['NDVI'],
    "min":-1,
    "max":1,
#"palette": ['#d73027', '#f46d43', '#fdae61','#fee08b', '#ffffbf',  '#d9ef8b','#a6d96a','#66bd63','#1a9850']
   "palette": ['#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718',
               '#74A901', '#66A000', '#529400', '#3E8601', '#207401', '#056201',
               '#004C00', '#023B01', '#012E01', '#011D01', '#011301']
}

ndvi=ndvi.getMapId(params[str(bandName)])
tiles=ndvi["tile_fetcher"].url_format
tiles=tiles.replace("https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/","https://keshton.ir/ee_maps/")
tiles

'https://keshton.ir/ee_maps/8e3c97613c5d02263d94ed3379c1173f-04755c8f87414b5db90a9efef1b9244c/tiles/{z}/{x}/{y}'

In [92]:
RGB = ['B4', 'B3', 'B2']
rgb_vis={
    'bands': ['B4', 'B3', 'B2'],
    "min":0,
    "max":6500,
   "palette": []
}
rgb_image= image.select(RGB)
rgb_tile=rgb_image.getMapId(rgb_vis)["tile_fetcher"].url_format
rgb_tile=rgb_tile.replace("https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/","https://keshton.ir/ee_maps/")
rgb_tile

'https://keshton.ir/ee_maps/b68aa93e114a41cbebabe830ac45e0c6-99575c12db99ef01245e3463d8d62197/tiles/{z}/{x}/{y}'

In [65]:
import folium

In [66]:
Map = folium.Map(location=[35.49035794928868,51.757017795358166],zoom_start=14)

basemaps = {'google sat':folium.TileLayer(
      tiles = 'http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}&s=Ga',
      attr = 'map data',
      name ='google satellite',
      overlay=True,
      control = True
)}

basemaps['google sat'].add_to(Map)

In [97]:
print(tiles)
folium.raster_layers.TileLayer(
      tiles = tiles,
      attr = 'map data',
      name ="ndvi map",
      overlay=True,
      control = True,
      opacity=1
  ).add_to(Map)



poly=folium.features.GeoJson(
  location,
  name = 'roads',
  control = True,
  style_function=lambda feature: {
        "fillColor": "#ffff00",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
    }
).add_to(Map)

https://keshton.ir/ee_maps/8e3c97613c5d02263d94ed3379c1173f-04755c8f87414b5db90a9efef1b9244c/tiles/{z}/{x}/{y}


In [99]:
Map.fit_bounds(poly.get_bounds())

folium.LayerControl().add_to(Map)
print(location)
Map

{'type': 'Feature', 'properties': {'name': 'test'}, 'geometry': {'coordinates': [[[51.75889330279665, 35.491301434102795], [51.75776151360827, 35.492283033293916], [51.75573167430704, 35.490740514895094], [51.75614994422264, 35.49032983942088], [51.75420621931508, 35.48912785036225], [51.75494434269825, 35.48828644732369], [51.75792144034125, 35.48972884713936], [51.75889330279665, 35.491301434102795]]], 'type': 'Polygon'}, 'id': 1}
